# Load All Data

In [1]:
import pickle as pkl

with open('./data/4_compiled/2024-06-13-15-23-21.pkl', 'rb') as f:
    datas = pkl.load(f)

In [2]:
datas.keys()

dict_keys(['coarse.pkl', 'coarse_wide.pkl', 'fine.pkl', 'low_temp.pkl', 'near_T1.pkl', 'near_T2.pkl'])

# Raw Plots

In [3]:
import matplotlib.pyplot as plt